In [8]:
import pandas as pd
import geopandas as gp
import networkx as nx
import osmnx as ox
import numpy as np
from osmnx.utils import log
from pyogrio import read_dataframe, write_dataframe
from shapely.geometry import shape, Point, LineString
from shapely.ops import nearest_points
import os
import matplotlib.pyplot as plt

gdf = read_dataframe("aquitaine-rail.gpkg", layer="node")

gdf.plot(markersize=4, color='red')
plt.title("Carte des nœuds du réseau ferroviaire en France")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

DataSourceError: aquitaine-rail.gpkg: No such file or directory